# Redes Generativas Adversarias (GAN)

Además de los Autoencoders, otra de las arquitecturas más reconocidas para resolver problemas de aprendizaje no supervisado son las **Redes Generativas Adversarias (GAN)**.

Las GANs fueron introducidas por Ian Goodfellow en 2014 y consisten en dos modelos que se entrenan simultáneamente: un **generador** y un **discriminador**. 

- El **generador** intenta crear datos falsos que sean indistinguibles de los datos reales a partir de, típicamente, un vector de ruido Gaussiano.
- El **discriminador** intenta distinguir entre datos reales y falsos resolviendo un problema de clasificación binaria.

El proceso de entrenamiento se denomina *"adversario"* puesto que el generador intenta engañar al discriminador, y el discriminador trata de no ser engañado.

El objetivo del generador es maximizar la probabilidad de que el discriminador clasifique sus muestras generadas como reales. 

Por otro lado, el objetivo del discriminador es minimizar su tasa de error en la clasificación de las muestras reales y generadas.

En esta práctica crearemos una **Deep Convolutional Generative Adversarial Network (DCGAN)** para generar nuevas imágenes de dígitos del conjunto *MNIST*.

Ten en cuenta que, aunque sea su uso más común, no todas las GAN tienen por que ser convolucionales o generar imágenes, también pueden generar datos estructurados o tabulares.

## Conjunto de datos

Utilizaremos de nuevo el conjunto [MNIST](https://es.wikipedia.org/wiki/Base_de_datos_MNIST). Como recordarás, este conjunto posee imágenes en escala de grises con números del 0 al 9 escritos a mano por personas. Cada una de estas imágenes tiene una resolución de $28 \times 28$ píxels. Al ser en escala de grises, cada imagen será un tensor de $1 \times 28 \times 28$.

Este conjunto de datos está pensado para resolver un problema de *Aprendizaje supervisado de multiclasificación*, pero en este caso descartaremos las etiquetas y utilizaremos solamente las imágenes.

De esta forma, nuestra red solo tendrá que aprender a generar imágenes de este dígito, y su tiempo de entrenamiento se reducirá considerablemente.

### Descargar conjunto

In [ ]:
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow.keras.datasets import mnist

# Fijar la semilla para obtener reproducibilidad
seed = 42
keras.utils.set_random_seed(seed)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Para simplificar el problema, nos quedaremos solamente con aquellas imágenes que tienen un dígito 8 o 3.

In [ ]:
import matplotlib.pyplot as plt

six_idxs = [idx for idx, v in enumerate(y_train) if v == 8 or v == 3]
x_train = x_train[six_idxs]
y_train = y_train[six_idxs]

# Mostramos la primera
plt.imshow(x_train[0],  cmap='gray')
plt.show()

Las imágenes vienen sin normalizar, por tanto lo primero será pasarlas del rango $[0,255]$ al rango $[-1,1]$ y luego crearemos los datasets.

In [ ]:
# Normalizar los datos al rango [-1, 1]
x_train, x_test = (x_train / 127.5) - 1, (x_test / 127.5) - 1

# Crear los datasets de tensorflow
train_data = tf.data.Dataset.from_tensor_slices((x_train, x_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, x_test))

## Arquitectura del modelo



**Red Generadora**

Esta red se encargará de transformar un vector de ruido Gaussiano en $\mathbb{R}^{100}$ en una imagen de $1 \times 28 \times 28$. Para ello se aplicarán de forma consecutiva una serie de convoluciones transpuestas (deconvoluciones) que generarán el volumen deseado.

> **NOTA:** Hay que tener en cuenta que la red generadora ha de crear imágenes en el mismo rango de valores que las imágenes reales. De no hacerlo, el discriminador tendría muy facil su tarea de detectar imágenes verdaderas y falsas.
En este caso hemos normalizado las imágenes del conjunto MNIST en el rango $[0, 1]$, por tanto pondremos una función de activación *sigmoide* a la salida de nuestra red para obtener el mismo resultado.

**Red Discriminadora**

La segunda red es la encargada de clasificar, dada una imagen de $1 \times 28 \times 28$, en verdadera o generada mediante clasificación binaria. Para ello aplicaremos de forma consecutiva una serie de convoluciones que irán transformando un volumen de activaciones en otro hasta llegar a una única neurona de salida.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_gan(learning_rate=1e-4):

    # Definición del generador
    input_tensor = layers.Input(shape=(100,))
    x = layers.Dense(7*7*128)(input_tensor)
    x = layers.ReLU()(x)
    x = layers.Reshape((7, 7, 128))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(1, kernel_size=3, padding='same', activation='tanh')(x)
    generator = models.Model(inputs=input_tensor, outputs=x, name="generator")

    # Definición del discriminador
    input_tensor = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(input_tensor)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(256, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    output_tensor = layers.Dense(1, activation="sigmoid")(x)
    discriminator = models.Model(inputs=input_tensor, outputs=output_tensor, name="discriminator")

    return generator, discriminator

### Entrenar GAN
Finalmente ya solo quedaría la parte más compleja, entrenar el modelo.
En este caso no podemos utilizar el método `fit` de Keras puesto que para cada batch tendremos que:

1. *Entrenar el discriminador:*
    * Se obtienen las predicciones del discriminador para los ejemplos del bath (los reales).
    * Se generan tantos vectores de ruido en $\mathbb{R}^{100}$ como ejemplos tenga el batch y se pasan por el generador **(congelando previamente sus pesos)**.
    * Se obtiene la loss del discriminador a partir de las predicciones del mismo ante los ejemplos reales y los falsos.
    * Optimizamos el discriminador.

2. *Entrenar el generador:*
    * Se generan nuevos vectores de ruido y se pasan por el generador. En este caso generamos $2*batch\_size$ para que el generador y el discriminador se optimicen con el mismo número de ejemplos.
    * Se obtiene la predicción del discriminador ante los ejemplos anteriores y su loss correspondiente forzando al discriminador a predecir 1.
    * Optimizamos el generador.


Ahora detallaremos los pasos a realizar para entrenar nuestro modelo. Antes vamos a crear una función que dibuje, tras cada época, 4 ejemplos utilizando el generador.

La otra función que crearemos es `train_step`. En esta definimos los pasos a realizar para cada batch de entrenamiento de nuestro modelo.

> **NOTA:** Esta función está decorada con `@tf.function`. Al hacerlo, TensorFlow compila todas las operaciones en un gráfico computacional y provoca que se ejecute mucho más rápido.

Finalmente definimos hiperparámetros, creamos el modelo, los datasets, la función de perdida, los optimizadores y entrenamos el modelo.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

# Función para mostrar imágenes
def plot_generated_images(generator, noise):
    num_images = noise.shape[0]
    generated_images = generator(noise).numpy()

    # Crear subgráficas
    fig, axes = plt.subplots(1, num_images, figsize=(10, 2))
    for i in range(num_images):
        axes[i].imshow(generated_images[i].squeeze(), cmap='gray')
        axes[i].axis('off')
    plt.show()

@tf.function
def train_step(real_images, generator, discriminator, noise_dim, cross_entropy, discriminator_optimizer, generator_optimizer):
    # Obtenemos el tamaño del batch actual (el último suele ser más pequeño)
    current_batch_size = real_images.shape[0]
    # Generar ruido aleatorio para crear imágenes falsas
    noise = tf.random.normal([current_batch_size, noise_dim])
    # Etiquetas para datos reales y generados
    real_labels = tf.ones((current_batch_size, 1))
    fake_labels = tf.zeros((current_batch_size, 1))

    # Generar imágenes falsas utilizando el generador
    generated_images = generator(noise) 

    # Entrenar el discriminador
    # -------------------------------------------------------------------------------------
    with tf.GradientTape() as disc_tape:
        # Obtenemos predicciones del discriminador para las imágenes reales y generadas
        real_output = discriminator(real_images)
        fake_output = discriminator(generated_images)
        # Calcular la pérdida (loss) del discriminador para imágenes reales y generadas
        disc_loss_real = cross_entropy(real_labels, real_output)
        disc_loss_fake = cross_entropy(fake_labels, fake_output)       
        # La pérdida total del discriminador es la suma de ambas pérdidas
        disc_loss = disc_loss_real + disc_loss_fake

    # Calcular los gradientes del discriminador con respecto a su pérdida
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    # Aplicar los gradientes al optimizador del discriminador
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Entrenar el generador
    # -------------------------------------------------------------------------------------
    
    with tf.GradientTape() as gen_tape:
        # Generar nuevas imágenes utilizando el ruido
        generated_images = generator(noise)
        # Obtener las predicciones del discriminador para las imágenes generadas
        fake_output = discriminator(generated_images)
        # Calcular la pérdida (loss) del generador usando las etiquetas reales
        gen_loss = cross_entropy(real_labels, fake_output)

    # Calcular los gradientes del generador con respecto a su pérdida
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    # Aplicar los gradientes al optimizador del generador
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    # Retornar las pérdidas del discriminador y del generador
    return disc_loss, gen_loss

# Configuración de parámetros 
noise_dim = 100
batch_size = 64
learning_rate = 0.0005
num_epochs = 50

# Crear y entrenar la GAN
generator, discriminator = create_gan()

# Definir los batches
train_dataset = train_data.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
test_dataset = test_data.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

# Función de pérdida
cross_entropy = tf.keras.losses.BinaryCrossentropy()

# Definir optimizadores
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.5)

# Generamos 6 vectores de ruido para monitorizar el generador durante el entrenamiento
plot_noise = tf.random.normal([6, noise_dim])

for epoch in range(num_epochs):
    for real_images, _ in train_dataset:
        # Entrenar el modelo
        disc_loss, gen_loss = train_step(real_images, generator, discriminator, noise_dim, cross_entropy, discriminator_optimizer, generator_optimizer)
        
    # Borrar la consola
    clear_output(wait=True)
    
    # Imprimir losses y gráfico
    print(f'Epoch {epoch}, Discriminator Loss: {disc_loss.numpy():0.3f}, Generator Loss: {gen_loss.numpy():0.3f}')
    plot_generated_images(generator, plot_noise)

## Generación
Para finalizar vamos a utilizar el generador para crear imágenes. En este caso vamos a crear imágenes asociadas a vectores de ruido $\mathbb{R}^{100}$ con todo valores -1, con todo valores 1 y $n$ intermedios.

In [ ]:
# Definir los valores para todos los elementos del vector de ruido
values = tf.linspace(-1.0, 1.0, 10)  # 10 valores entre -1 y 1

# Crear una figura para los resultados, una sola fila con 5 columnas
fig, axes = plt.subplots(1, len(values), figsize=(15, 3))

# Generar imágenes para cada valor en el vector de ruido
for i, v in enumerate(values):
    # Crear un vector de ruido donde todos los elementos son iguales al valor actual
    noise = tf.fill([1, noise_dim], v)
    # Generar una imagen con el generador
    generated_image = generator(noise)
    
    # Plotear la imagen en la posición correcta de la cuadrícula
    ax = axes[i]
    ax.axis('off')
    ax.imshow(tf.squeeze(generated_image), cmap='Greys_r')
    ax.set_title(f'{v:.2f}', fontsize=10)

plt.tight_layout()
plt.show()